In [1]:
import numpy as np
import os
import onnx
from onnxruntime_extensions import get_library_path, PyOp, onnx_op, PyOrtFunction
import onnxruntime as ort
from qonnx.core.datatype import DataType
from qonnx.core.modelwrapper import ModelWrapper
from onnx.helper import make_attribute
from driver_base import FINNExampleOverlay
from driver import io_shape_dict
set_pyop = False

platform = "zynq-iodma"
batch_size = 1
bitfile = "finn-accel.bit"
outputfile = "output.npy"
runtime_weight_dir = "runtime_weights/"

# instantiate FINN accelerator driver and pass batchsize and bitfile
accel = FINNExampleOverlay(
    bitfile_name = bitfile, platform = platform,
    io_shape_dict = io_shape_dict, batch_size = batch_size,
    runtime_weight_dir = runtime_weight_dir
)

if not set_pyop:
    # Implement the CustomOp by decorating a function with onnx_op
    @onnx_op(op_type="StreamingDataflowPartition", inputs=[PyOp.dt_float], outputs=[PyOp.dt_float])
    def StreamingDataflowPartition(inputs):
        obuf_normal = accel.execute(inputs)
        return obuf_normal.astype(np.float32)
    set_pyop = True
print(set_pyop)

True


## HW session for the CONV_10 split Partition 0

In [29]:
inp = np.load("dog.npy")

model_func = PyOrtFunction.from_model("./dataflow_parent_updated.onnx")
outputs = model_func(inp)

## CPU session for CONV_10 split Partition 0

In [28]:
from qonnx.transformation.double_to_single_float import DoubleToSingleFloat
from onnx import helper

def set_multithreshold_default(model):#,save_model):
    '''
    Pass a modelproto model and the save file
    '''
    model = model.transform(DoubleToSingleFloat())
    new_attr = [helper.make_attribute("out_scale", 1.0),
                helper.make_attribute("out_bias", 0.0),
                helper.make_attribute("data_layout","NCHW")]

    for n in model.graph.node:
        if n.op_type == "MultiThreshold":
            out_scale,bias,datalayout = False,False,False
            for na in n.attribute:
                if na.name == "out_scale": out_scale = True
                if na.name == "out_bias": bias = True
                if na.name == "data_layout": datlayout = True
            if not out_scale: n.attribute.append(new_attr[0])
            if not bias: n.attribute.append(new_attr[1])
            if not datalayout: n.attribute.append(new_attr[2])

            n.domain = "ai.onnx.contrib"
#     model.save(save_model)
    return model

model = ModelWrapper("./partition_0.onnx")
model = set_multithreshold_default(model)
sess = ort.InferenceSession(model.model.SerializeToString(),so)

inp_name = [i.name for i in sess.get_inputs()]
inp_shape = [i.shape for i in sess.get_inputs()]

x = np.load('dog.npy')

res = sess.run([],{inp_name[0]:x})
y = np.array(res)
y.shape

(1, 1, 512, 14, 14)

## Compare the two outputs

In [40]:
equal = np.testing.assert_array_almost_equal(y[0],outputs)
if equal == None:
    print("CPU and HW output are the same!")

CPU and HW output are the same!
